# Beginning

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import darts
from darts import TimeSeries
from darts.models import NaiveSeasonal, NaiveMean, NaiveDrift
from darts.models import StatsForecastAutoARIMA, StatsForecastAutoETS, StatsForecastAutoCES, RNNModel, ExponentialSmoothing, BlockRNNModel
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from tqdm import tqdm_notebook as tqdm
from darts.dataprocessing.transformers import Scaler
from darts.models import RNNModel, ExponentialSmoothing, BlockRNNModel
from darts.metrics import mape, mase, mse, rmse, ase, ape, r2_score, smape
from darts.utils.statistics import check_seasonality, plot_acf
from darts.datasets import AirPassengersDataset, SunspotsDataset
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dropout, Dense, Flatten
import warnings
warnings.filterwarnings("ignore")
import logging
logging.disable(logging.CRITICAL)
import random
from tensorflow.keras.optimizers import Adam, RMSprop, SGD, Nadam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error, mean_absolute_error
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
import tensorflow as tf
from tensorflow.keras import backend as K
import os

from hyperopt import base
os.environ['PYTHONHASHSEED'] = '0'
os.environ['HYPEROPT_FMIN_SEED'] = "1"
random.seed(88)
np.random.seed(88)
tf.random.set_seed(88)
base.have_been_bugged = False  
rstate = np.random.default_rng(88)

python(54939) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
/Users/jacobzhao/anaconda3/envs/prophet/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.
2024-06-15 15:58:14.778185: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
df_total = pd.read_csv('time_series_thesis_question_1.csv', index_col=0)
df_total.index = pd.to_datetime(df_total.index)

In [3]:
df = TimeSeries.from_dataframe(df_total)

In [4]:
train_1, val_1 = df.split_before(pd.Timestamp('20230101'))
train_2, val_2 = df.split_before(pd.Timestamp('20230401'))
train_3, val_3 = df.split_before(pd.Timestamp('20230701'))

In [5]:
def Xy(sliding_windows):
    X = [[list(window[:-1]) for window in windows] for windows in sliding_windows]
    y = [[window[-1] for window in windows] for windows in sliding_windows]
    return np.array(X), np.array(y)

In [6]:
from sklearn.metrics import mean_absolute_percentage_error, mean_absolute_error

def spliter(df_total,
            k = 4,
            test_size = 3,
            val_size = 3):
    test = []
    val = []
    train = []
    NN_sets = {}
    window_size = k+1


    for col in df_total.columns:
        windows = [np.array(window) for window in df_total[col].rolling(window_size) if len(window) == window_size]
        test.append(windows[-(test_size):])
        val.append(windows[-(test_size+val_size):-(test_size)])
        train.append(windows[:-(test_size+val_size)])
    NN_sets['X_train'], NN_sets['y_train'] = Xy(train)
    NN_sets['X_val'], NN_sets['y_val'] = Xy(val)
    NN_sets['X_test'], NN_sets['y_test'] = Xy(test)
    return NN_sets


In [7]:
import darts
def NN_metricker(y_pred):
  y_pred_df = pd.DataFrame(y_pred.reshape((-1, 3)).transpose())
  y_pred_df.columns = df_total.columns
  y_pred_df.index = df_total.index[-3:]
  y_pred_df.index = pd.to_datetime(y_pred_df.index)
  y_pred_tf = TimeSeries.from_dataframe(y_pred_df)
  SMAPE = darts.metrics.smape(val_1, y_pred_tf)
  MASE = darts.metrics.mase(val_1, y_pred_tf, train_1)
  MAE = darts.metrics.mae(val_1, y_pred_tf)
  print(
      "Symmetric Mean absolute percentage error: {:.2f}%.".format(
          SMAPE),
          "MASE: {:.2f}".format(MASE),
          "MAE: {:.2f}".format(MAE)
      )
  return y_pred_df, SMAPE, MASE, MAE


In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, LSTM, Dense
from tensorflow.keras.losses import Huber
def create_cnn_lstm_model(window, lstm_layers, n_filters, optimizer_name, lr, kernel_size):
    NN_sets = spliter(df_total, k=window) 

    model = Sequential()
    
    # CNN part
    model.add(Conv1D(filters=n_filters, kernel_size=kernel_size, activation='relu'))
    
    for _ in range(lstm_layers):
        model.add(LSTM(n_filters, return_sequences=True))
    
    # LSTM part
    model.add(LSTM(n_filters // 2, return_sequences=False))
    
    # Output layer
    model.add(Dense(1))
    
    optimizer_class = {'adam': Adam, 'rmsprop': RMSprop, 'sgd': SGD, 'nadam': Nadam}[optimizer_name]
    optimizer = optimizer_class(lr)
    model.compile(optimizer=optimizer, loss=Huber())
    
    return model, NN_sets

# Tuning hyperparameters

In [15]:
import pandas as pd
import numpy as np
import tensorflow as tf
import random
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, LSTM, Dense
from tensorflow.keras.optimizers import Adam, RMSprop, SGD, Nadam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error, mean_absolute_error
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from tensorflow.keras import backend as K

K.clear_session()    
# Set random seeds
seed = 88
np.random.seed(seed)
tf.random.set_seed(seed)
random.seed(seed)

# Hyperparameters to tune
n_lstm_layers_values = [1, 2]
n_filters_values = [8, 16, 32, 64, 128]
optimizer_values = ['adam', 'rmsprop', 'nadam']
window_values = [16, 20, 24, 28]
kernel_size_values = [3, 5, 7, 9]
lr_values = list(np.arange(1e-4, 11e-4, 1e-4))
trials_results = []

# Define the objective function for Hyperopt
def objective(params):
    window = int(params['window'])
    n_lstm_layers = int(params['n_lstm_layers'])
    n_filters = int(params['n_filters'])
    optimizer_name = params['optimizer']
    lr = params['lr']
    kernel_size = int(params['kernel_size'])

    # Create the CNN-LSTM model with the given hyperparameters
    model, NN_sets = create_cnn_lstm_model(window, n_lstm_layers, n_filters, optimizer_name, lr, kernel_size)
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    X_train = NN_sets['X_train'].reshape((-1, window, 1))
    y_train = NN_sets['y_train'].reshape((-1, 1))
    X_val = NN_sets['X_val'].reshape((-1, window, 1))
    y_val = NN_sets['y_val'].reshape((-1, 1))

    history = model.fit(
        X_train, y_train,
        epochs=30,
        batch_size=16,
        validation_data=(X_val, y_val),
        callbacks=[early_stopping],
        verbose=0
    )

    val_loss = history.history['val_loss'][-1]
    trials_results.append({'params': params, 'val_loss': val_loss})
    return {'loss': val_loss, 'status': STATUS_OK}
# Define the hyperparameter search space

search_space = {
    'window': hp.choice('window', window_values),
    'n_lstm_layers': hp.choice('n_lstm_layers', n_lstm_layers_values),
    'n_filters': hp.choice('n_filters', n_filters_values),
    'optimizer': hp.choice('optimizer', optimizer_values),
    'lr': hp.choice('lr', lr_values),
    'kernel_size': hp.choice('kernel_size', kernel_size_values)
}
# Conduct the Bayesian optimization
trials = Trials()
best = fmin(objective, search_space, algo=tpe.suggest, max_evals=100, trials=trials)

# Print the best hyperparameters and the corresponding validation loss
print('Best hyperparameters:')
print(best)

  0%|          | 0/100 [00:00<?, ?trial/s, best loss=?]

2024-06-15 17:02:33.189582: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


100%|██████████| 100/100 [6:31:29<00:00, 234.90s/trial, best loss: 0.9143925309181213]   
Best hyperparameters:
{'kernel_size': 0, 'lr': 8, 'n_filters': 1, 'n_lstm_layers': 0, 'optimizer': 2, 'window': 0}


In [17]:
import os
import pandas as pd

# Assuming trials_results is your list of dictionaries
df = pd.DataFrame(trials_results)

# If you want to flatten the 'params' column into separate columns
df = pd.concat([df.drop(['params'], axis=1), df['params'].apply(pd.Series)], axis=1)

# Create directory if it doesn't exist
if not os.path.exists('CNNLSTM_results'):
    os.makedirs('CNNLSTM_results')

# Write df to a csv file in the specified directory
df.to_csv('CNNLSTM_results/CNNLSTM_tuning_results.csv', index=False)

In [19]:
K.clear_session()
tf.keras.backend.clear_session()

best_window = window_values[best['window']]
best_n_lstm_layers = n_lstm_layers_values[best['n_lstm_layers']]
best_n_filters = n_filters_values[best['n_filters']]
best_optimizer_name = optimizer_values[best['optimizer']]
best_lr = lr_values[best['lr']]
best_kernel_size = kernel_size_values[best['kernel_size']]

print(f"Best window size: {best_window}")
print(f"Best number of LSTM layers: {best_n_lstm_layers}")
print(f"Best number of filters: {best_n_filters}")
print(f"Best optimizer: {best_optimizer_name}")
print(f"Best learning rate: {best_lr}")
print(f"Best kernel size: {best_kernel_size}")

best_model, NN_sets = create_cnn_lstm_model(best_window, best_n_lstm_layers, best_n_filters, best_optimizer_name, best_lr, best_kernel_size)
early_stop = EarlyStopping(monitor='val_loss', patience=5)

X_train = NN_sets['X_train'].reshape((-1, best_window, 1))
y_train = NN_sets['y_train'].reshape((-1, 1))
X_val = NN_sets['X_val'].reshape((-1, best_window, 1))
y_val = NN_sets['y_val'].reshape((-1, 1))

history = best_model.fit(
    X_train, y_train,
    epochs=30,
    batch_size=16,
    validation_data=(X_val, y_val),
    callbacks=[early_stop],
    verbose=0
)

Best window size: 16
Best number of LSTM layers: 1
Best number of filters: 16
Best optimizer: nadam
Best learning rate: 0.0009000000000000001
Best kernel size: 3


In [20]:
y_pred = best_model.predict(NN_sets['X_test'].reshape((-1, best_window, 1)))
y_pred_df, SMAPE, MASE, MAE = NN_metricker(y_pred)

73/73 [==============================] - 1s 3ms/step
Symmetric Mean absolute percentage error: 14.65%. MASE: 1.14 MAE: 1.14


In [22]:
SMAPE_values = []
MASE_values = []
MAE_values = []

for i in range(20):
    K.clear_session()
    tf.keras.backend.clear_session()
    best_model, NN_sets = create_cnn_lstm_model(best_window, best_n_lstm_layers, best_n_filters, best_optimizer_name, best_lr, best_kernel_size)    
    early_stop = EarlyStopping(monitor='val_loss', patience=5)
    history = best_model.fit(
        NN_sets['X_train'].reshape((-1, best_window, 1)),
        NN_sets['y_train'].reshape((-1, 1)),
        epochs=30,
        batch_size=16,
        validation_data=(NN_sets['X_val'].reshape((-1, best_window, 1)), NN_sets['y_val'].reshape((-1, 1))),
        callbacks=[early_stop],
        verbose=0
    )
    y_pred = best_model.predict(NN_sets['X_test'].reshape((-1, best_window, 1)))
    y_pred_df, SMAPE, MASE, MAE = NN_metricker(y_pred)
    
    # Append the metrics to the lists
    SMAPE_values.append(SMAPE)
    MASE_values.append(MASE)
    MAE_values.append(MAE)


73/73 [==============================] - 3s 4ms/step
Symmetric Mean absolute percentage error: 14.79%. MASE: 1.14 MAE: 1.15
73/73 [==============================] - 4s 4ms/step
Symmetric Mean absolute percentage error: 14.50%. MASE: 1.12 MAE: 1.13
73/73 [==============================] - 2s 4ms/step
Symmetric Mean absolute percentage error: 14.61%. MASE: 1.14 MAE: 1.15
73/73 [==============================] - 2s 3ms/step
Symmetric Mean absolute percentage error: 14.49%. MASE: 1.13 MAE: 1.15
73/73 [==============================] - 2s 2ms/step
Symmetric Mean absolute percentage error: 14.53%. MASE: 1.14 MAE: 1.14
73/73 [==============================] - 2s 4ms/step
Symmetric Mean absolute percentage error: 14.51%. MASE: 1.12 MAE: 1.14
73/73 [==============================] - 2s 4ms/step
Symmetric Mean absolute percentage error: 14.49%. MASE: 1.12 MAE: 1.14
73/73 [==============================] - 15s 3ms/step
Symmetric Mean absolute percentage error: 14.54%. MASE: 1.12 MAE: 1.15
73/73 [

In [23]:
import os
import pickle

with open('CNNLSTM_results/CNNLSTM_SMAPE_values.pkl', 'wb') as f:
    pickle.dump(SMAPE_values, f)
with open('CNNLSTM_results/CNNLSTM_MASE_values.pkl', 'wb') as f:
    pickle.dump(MASE_values, f)
with open('CNNLSTM_results/CNNLSTM_MAE_values.pkl', 'wb') as f:
    pickle.dump(MAE_values, f)